In [11]:
### Import:

In [12]:
import pandas as pd
import numpy as np


### Caricamento del dataset

In [13]:
try:
    df = pd.read_csv("./data/monsters.csv")
    print("✅ Dataset caricato con successo.")
except FileNotFoundError:
    print("❌ Errore: File non trovato.")
    raise


✅ Dataset caricato con successo.


### --- PULIZIA PRELIMINARE---
Prima di creare sub_race, dobbiamo assicurarci che 'subtype' non contenga valori inutili come "any race".
Se li contiene, li trasformiamo in NaN (Null), così la logica successiva potrà riempirli con il Type.

In [14]:
print("--- Pulizia valori ambigui ---")

# Lista di termini che vogliamo considerare come "Vuoti"
junk_terms = ['any race', 'any', 'varies']

# Controlliamo la colonna subtype
if 'subtype' in df.columns:
    for term in junk_terms:
        # Trova le righe dove subtype contiene il termine spazzatura (es. "any race")
        mask = df['subtype'].astype(str).str.contains(term, case=False, na=False)
        
        count = mask.sum()
        if count > 0:
            # Sostituisci con NaN (valore nullo reale)
            df.loc[mask, 'subtype'] = np.nan
            print(f"🧹 Trovati e rimossi {count} casi di '{term}' in subtype.")


--- Pulizia valori ambigui ---
🧹 Trovati e rimossi 28 casi di 'any race' in subtype.


### --- LOGICA DI AGGREGAZIONE ---
1. Prendo **group**.
2. Se group manca, prendo **subtype** (ora pulito da "any race").
3. Se mancano entrambi, prendo **type**.

In [15]:
# 1. Base: Group
df['sub_race'] = df['group']

# 2. Fill con Subtype (i NaN e gli ex-"any race" vengono riempiti qui o saltati)
df['sub_race'] = df['sub_race'].fillna(df['subtype'])

# 3. Fill con Type (Fallback finale)
df['sub_race'] = df['sub_race'].fillna(df['type'])

print("✅ Colonna 'sub_race' creata correttamente.")


✅ Colonna 'sub_race' creata correttamente.


### --- RIMIZIONE COLONNE VECCHIE ---
Ora rimuoviamo group, subtype e desc (se presente).

In [16]:
cols_to_drop = ['group', 'subtype', 'desc']
# Filtriamo solo quelle che esistono davvero per evitare errori
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]

if existing_cols_to_drop:
    df = df.drop(columns=existing_cols_to_drop)
    print(f"🗑️ Colonne rimosse: {existing_cols_to_drop}")


🗑️ Colonne rimosse: ['group', 'subtype', 'desc']


### --- RIORDINAMENTO COLONNE ---
Spostiamo 'sub_race' dopo 'type'.

In [17]:
cols = df.columns.tolist()

if 'sub_race' in cols and 'type' in cols:
    cols.remove('sub_race')
    type_index = cols.index('type')
    cols.insert(type_index + 1, 'sub_race')
    df = df[cols]
    print("abc Ordine colonne sistemato.")


abc Ordine colonne sistemato.


### --- VERIFICA FINALE ---
Controlliamo se esistono ancora "any race" nella nuova colonna.

In [18]:
test_junk = df[df['sub_race'].astype(str).str.contains("any race", case=False)]
if len(test_junk) == 0:
    print("✨ PERFETTO: Nessun 'any race' trovato in sub_race.")
else:
    print(f"⚠️ ATTENZIONE: Ci sono ancora {len(test_junk)} righe con 'any race'.")
    print(test_junk[['name', 'type', 'sub_race']].head())


✨ PERFETTO: Nessun 'any race' trovato in sub_race.


### --- RIORDINAMENTO COLONNE ---
Spostiamo 'sub_race' dopo 'type'.

In [ ]:
output_path = "./data/monsters.csv"
try:
    df.to_csv(output_path, index=False)
    print(f"\n💾 Salvataggio completato in: {output_path}")
except PermissionError:
    print("❌ Errore: File aperto. Chiudilo e riprova.")
